In [1]:
import pandas as pd
import csv
import os

In [16]:
#os.chdir("C:\\Users\\Louis\\Desktop\\NBA project\\2000never\\")
os.chdir("C:\\Users\\Louis\\Desktop\\NBA project\\2016-17\\")

In [17]:
dirName=os.getcwd()

In [18]:
dirName

'C:\\Users\\Louis\\Desktop\\NBA project\\2016-17'

In [19]:
listOfFile = os.listdir(dirName)

In [20]:
print(listOfFile)

['[2016-10-25]-0021600001-NYK@CLE.csv', '[2016-10-25]-0021600002-UTA@POR.csv', '[2016-10-25]-0021600003-SAS@GSW.csv', '[2016-10-26]-0021600004-MIA@ORL.csv', '[2016-10-26]-0021600005-DAL@IND.csv', '[2016-10-26]-0021600006-BKN@BOS.csv', '[2016-10-26]-0021600007-DET@TOR.csv', '[2016-10-26]-0021600008-CHA@MIL.csv', '[2016-10-26]-0021600009-MIN@MEM.csv', '[2016-10-26]-0021600010-DEN@NOP.csv', '[2016-10-26]-0021600011-OKC@PHI.csv', '[2016-10-26]-0021600012-SAC@PHX.csv', '[2016-10-26]-0021600013-HOU@LAL.csv', '[2016-10-27]-0021600014-WAS@ATL.csv', '[2016-10-27]-0021600015-BOS@CHI.csv', '[2016-10-27]-0021600016-SAS@SAC.csv', '[2016-10-27]-0021600017-LAC@POR.csv', '[2016-10-28]-0021600018-CLE@TOR.csv', '[2016-10-28]-0021600019-IND@BKN.csv', '[2016-10-28]-0021600020-ORL@DET.csv', '[2016-10-28]-0021600021-PHX@OKC.csv', '[2016-10-28]-0021600022-CHA@MIA.csv', '[2016-10-28]-0021600023-HOU@DAL.csv', '[2016-10-28]-0021600024-LAL@UTA.csv', '[2016-10-28]-0021600025-GSW@NOP.csv', '[2016-10-29]-0021600026

In [21]:
newdirName=dirName+"new"

In [22]:
newdirName

'C:\\Users\\Louis\\Desktop\\NBA project\\2016-17new'

In [23]:
for entry in listOfFile:
    fullPath = dirName  + "/" + entry
    df=pd.read_csv(fullPath)
    away_run_counter=0
    home_run_counter=0
    for i in (range(2, len(df.index))):
        df.at[i, 'Away_points_scored']=(df.at[i, 'away_score'])-(df.at[i-1, 'away_score'])
        df.at[i, 'Home_points_scored']=(df.at[i, 'home_score'])-(df.at[i-1, 'home_score'])
    for i in (range(2, len(df.index))):
        if df.at[i, 'Away_points_scored']==0 and df.at[i, 'Home_points_scored']==0:
            away_run_counter+=0
            home_run_counter+=0
            df.at[i, 'away_run']= away_run_counter
            df.at[i, 'home_run']= home_run_counter
        elif df.at[i, 'Away_points_scored']>0 and df.at[i, 'Home_points_scored']==0:
            away_run_counter+=df.at[i, 'Away_points_scored']
            home_run_counter=0
            df.at[i, 'away_run']= away_run_counter
            df.at[i, 'home_run']= home_run_counter
        elif df.at[i, 'Away_points_scored']==0 and df.at[i, 'Home_points_scored']>0:
            home_run_counter+=df.at[i, 'Home_points_scored']
            away_run_counter=0
            df.at[i, 'away_run']= away_run_counter
            df.at[i, 'home_run']= home_run_counter
    df['total_run']=df['home_run']+df['away_run']
    df=df[(df['type']==' timeout: short') | 
                             (df['type']==' timeout: regular')|
                             (df['type']==' timeout: official')|
                             (df['type']==' timeout: no timeout')|
                             (df['Home_points_scored']!= 0)| 
                             (df['Away_points_scored']!= 0)]
    df=df.reset_index()
    df['subsequent_nontvto']=0
    for i in (range(0, len(df.index)-1)):
        if df.loc[i+1, 'type']==' timeout: short' or df.loc[i+1, 'type']==' timeout: regular' or df.loc[i+1, 'type']==' timeout: no timeout':
            df.loc[i, 'subsequent_nontvto']=1
        else: 
            df.loc[i, 'subsequent_nontvto']=0
            
    df['subsequent_tv_to']=0
    for i in (range(0, len(df.index)-1)):
        if df.loc[i+1, 'type']==' timeout: official':
            df.loc[i, 'subsequent_tv_to']=1
        else: 
            df.loc[i, 'subsequent_tv_to']=0
            
    #Here onward is Kevin's code for the rolling sum of scores
    
    df['remaining_time_in_game'] = pd.to_datetime(df['remaining_time']) + pd.to_timedelta((4 - df['period']) * 12., unit='m')
    
    Rev = df[::-1] # reverse row order

# Calculate rolling sum - points in *previous* 3 minutes of *time remaining* equivalent to *next* 3 minutes of
# *elapsed time*
    Rev.loc[:,'home_points_lookahead_run'] = Rev[['Home_points_scored', 'remaining_time_in_game']].rolling(window='180s',
                                                                              on='remaining_time_in_game', closed='left').sum()['Home_points_scored']
    Rev.loc[:,'away_points_lookahead_run'] = Rev[['Away_points_scored', 'remaining_time_in_game']].rolling(window='180s',
                                                                              on='remaining_time_in_game', closed='left').sum()['Away_points_scored']

    df_out = Rev[::-1]

    df_out.to_csv(newdirName  + "/"+"new" + entry)

C:\Users\Louis\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Louis\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
